[Python/pdfquery: Scraping the FIFA World Player of the Year votes](http://www.markhneedham.com/blog/2015/01/22/pythonpdfquery-scraping-the-fifa-world-player-of-the-year-votes-pdf-into-shape/) (here: compare with `pdftotext` and text processing)

In [1]:
import os
import io
import re
import sys
import csv
import urllib
import subprocess
if sys.version_info.major == 2:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

URL = 'https://raw.githubusercontent.com/mneedham/fifa/master/fboaward_menplayer2014_neutral.pdf'
PDF = URL.rpartition('/')[2]
TXT = '%s.txt' % os.path.splitext(PDF)[0]
CSV = '%s.csv' % os.path.splitext(PDF)[0]

if not os.path.exists(PDF):
    urlretrieve(URL, PDF)

if not os.path.exists(TXT):  # requires one of popper-utils, miktex-poppler-bin, xpdf
    subprocess.check_call(['pdftotext', '-layout', PDF, TXT])

def load_pages(filename=TXT, encoding='utf-8', delimiter='\f'):
    with io.open(filename, encoding=encoding) as f:
        text = f.read()
    return text.split(delimiter)

pages = load_pages()

len(pages)

18

In [2]:
def iterrows(pages):
    first = None
    for p in pages:
        if not p:
            continue
        title, _, rest = p.partition('\n\n\n')
        header, _, rest = rest.partition('\n')
        header = re.split(r' {2,}', header)
        if first is None:
            yield header
            first = title, header
        else:
            assert title, header == first
        table, _, footer = rest.partition('\n                              ')
        table, footer = (s.strip() for s in (table, footer))
        page, total = map(int, re.match(r'(\d+) / (\d+)$', footer).groups())
        for line in table.splitlines():
            if not line:
                continue
            if line.startswith(' '):
                assert line.endswith(' Grenadines')
                continue
            row = re.split(r' {2,}', line)
            assert len(row) == len(header)
            if row[1] == 'St. Vincent and the':
                row[1] += ' Grenadines'
            yield row

rows = list(iterrows(pages))
rows[:2]

[['Vote',
  'Country',
  'Name',
  'First (5 points)',
  'Second (3 points)',
  'Third (1 point)'],
 ['Captain',
  'Afghanistan',
  'Amiri Islam',
  'Messi Lionel',
  'Cristiano Ronaldo',
  'Ibrahimovic Zlatan']]

In [3]:
rows[163]

['Captain',
 'Sweden',
 'Ibrahimovic Zlatan',
 'Messi Lionel',
 'Neuer Manuel',
 'Cristiano Ronaldo']

In [4]:
def write_csv(rows, filename=CSV, encoding='utf-8', dialect=csv.excel, has_header=True):
    py2 = (sys.version_info.major == 2)
    open_kwargs = {'mode': 'wb'} if py2 else \
                  {'mode': 'w', 'encoding': encoding, 'newline': ''}
    with io.open(filename, **open_kwargs) as f:
        writer = csv.writer(f, dialect=dialect)
        if py2:
            writerow = lambda r: writer.writerow([v.encode(encoding) for v in r])
        else:
            writerow = writer.writerow
        for ndata, r in enumerate(rows, 0 if has_header else 1):
            writerow(r)
    return ndata

write_csv(rows)

544

In [5]:
import pandas as pd

df = pd.read_csv(CSV, encoding='utf-8', dialect=csv.excel)

df.head()

,Vote,Country,Name,First (5 points),Second (3 points),Third (1 point)
0,Captain,Afghanistan,Amiri Islam,Messi Lionel,Cristiano Ronaldo,Ibrahimovic Zlatan
1,Captain,Albania,Cana Lorik,Cristiano Ronaldo,Robben Arjen,Mueller Thomas
2,Captain,Algeria,Bougherra Madjid,Cristiano Ronaldo,Robben Arjen,Benzema Karim
3,Captain,American Samoa,Luvu Rafe Talalelei,Neymar,Robben Arjen,Cristiano Ronaldo
4,Captain,Andorra,Sonejee Masand Oscar,Cristiano Ronaldo,Mueller Thomas,Kroos Toni


In [6]:
df['Vote'].value_counts().to_frame('Votes').sort_index()

,Votes
Captain,182
Coach,181
Media,181


In [7]:
pd.set_option('display.max_rows', 250)

df.groupby('Country')['Vote'].value_counts().unstack(fill_value=0)

Vote,Captain,Coach,Media
Country,,,
Afghanistan,1,1,0
Albania,1,1,1
Algeria,1,1,1
American Samoa,1,1,0
Andorra,1,1,1
Angola,1,1,1
Anguilla,1,1,0
Antigua and Barbuda,1,1,1
Argentina,1,1,1


In [8]:
df.head()

,Vote,Country,Name,First (5 points),Second (3 points),Third (1 point)
0,Captain,Afghanistan,Amiri Islam,Messi Lionel,Cristiano Ronaldo,Ibrahimovic Zlatan
1,Captain,Albania,Cana Lorik,Cristiano Ronaldo,Robben Arjen,Mueller Thomas
2,Captain,Algeria,Bougherra Madjid,Cristiano Ronaldo,Robben Arjen,Benzema Karim
3,Captain,American Samoa,Luvu Rafe Talalelei,Neymar,Robben Arjen,Cristiano Ronaldo
4,Captain,Andorra,Sonejee Masand Oscar,Cristiano Ronaldo,Mueller Thomas,Kroos Toni


In [9]:
pf = df.melt('Vote', df.columns[-3:].tolist(), var_name='Rank', value_name='Name')\
    .assign(Points=lambda x: x['Rank'].str.extract(r'(\d+)', expand=False).astype(int))\
    .pivot_table('Points', ['Vote', 'Name'], 'Rank', aggfunc=[len, sum], fill_value=0)\
    .assign(Total=lambda x: x['sum'].sum(axis=1))

pf.sum(level='Name').sort_values(by='Total', ascending=False)

len                                    \
Rank                   First (5 points) Second (3 points) Third (1 point)   
Name                                                                        
Cristiano Ronaldo                   303                96              41   
Messi Lionel                         55               132             101   
Neuer Manuel                         85                89              77   
Robben Arjen                         17                69              59   
Mueller Thomas                       22                35              50   
Lahm Philipp                         12                17              31   
Neymar                                6                13              39   
Rodriguez James                       6                 8              18   
Kroos Toni                            5                11              12   
Di Maria Angel                        4                 8              19   
Costa Diego                           2                 9              13   
Bale Gareth                           5                 6               6   
Ibrahimovic Zlatan                    3                 8              10   
Toure Yaya                            0                 9              15   
Goetze Mario                          5                 5               1   
Benzema Karim                         1                 7              11   
Iniesta Andres                        3                 4               6   
Schweinsteiger Bastian                2                 3               9   
Mascherano Javier                     2                 4               5   
Courtois Thibaut                      4                 1               2   
Hazard Eden                           0                 4              11   
Pogba Paul                            1                 4               3   
Ramos Sergio                          1                 2               5   

                                    sum                                    \
Rank                   First (5 points) Second (3 points) Third (1 point)   
Name                                                                        
Cristiano Ronaldo                  1515               288              41   
Messi Lionel                        275               396             101   
Neuer Manuel                        425               267              77   
Robben Arjen                         85               207              59   
Mueller Thomas                      110               105              50   
Lahm Philipp                         60                51              31   
Neymar                               30                39              39   
Rodriguez James                      30                24              18   
Kroos Toni                           25                33              12   
Di Maria Angel                       20                24              19   
Costa Diego                          10                27              13   
Bale Gareth                          25                18               6   
Ibrahimovic Zlatan                   15                24              10   
Toure Yaya                            0                27              15   
Goetze Mario                         25                15               1   
Benzema Karim                         5                21              11   
Iniesta Andres                       15                12               6   
Schweinsteiger Bastian               10                 9               9   
Mascherano Javier                    10                12               5   
Courtois Thibaut                     20                 3               2   
Hazard Eden                           0                12              11   
Pogba Paul                            5                12               3   
Ramos Sergio                          5                 6               5   

                       Total  

In [10]:
pf.loc['Captain'].sort_values(by='Total', ascending=False).head()

len                                    \
Rank              First (5 points) Second (3 points) Third (1 point)   
Name                                                                   
Cristiano Ronaldo               99                37              13   
Messi Lionel                    31                44              25   
Neuer Manuel                    22                13              27   
Robben Arjen                     3                25              22   
Mueller Thomas                   3                11              16   

                               sum                                   Total  
Rank              First (5 points) Second (3 points) Third (1 point)        
Name                                                                        
Cristiano Ronaldo              495               111              13   619  
Messi Lionel                   155               132              25   312  
Neuer Manuel                   110                39              27   176  
Robben Arjen                    15                75              22   112  
Mueller Thomas                  15                33              16    64

In [11]:
pf.loc['Coach'].sort_values(by='Total', ascending=False).head()

len                                    \
Rank              First (5 points) Second (3 points) Third (1 point)   
Name                                                                   
Cristiano Ronaldo               89                31              14   
Messi Lionel                    20                49              35   
Neuer Manuel                    32                31              25   
Robben Arjen                    10                26              20   
Mueller Thomas                   8                 8              17   

                               sum                                   Total  
Rank              First (5 points) Second (3 points) Third (1 point)        
Name                                                                        
Cristiano Ronaldo              445                93              14   552  
Messi Lionel                   100               147              35   282  
Neuer Manuel                   160                93              25   278  
Robben Arjen                    50                78              20   148  
Mueller Thomas                  40                24              17    81

In [12]:
pf.loc['Media'].sort_values(by='Total', ascending=False).head()

len                                    \
Rank              First (5 points) Second (3 points) Third (1 point)   
Name                                                                   
Cristiano Ronaldo              115                28              14   
Neuer Manuel                    31                45              25   
Messi Lionel                     4                39              41   
Mueller Thomas                  11                16              17   
Robben Arjen                     4                18              17   

                               sum                                   Total  
Rank              First (5 points) Second (3 points) Third (1 point)        
Name                                                                        
Cristiano Ronaldo              575                84              14   673  
Neuer Manuel                   155               135              25   315  
Messi Lionel                    20               117              41   178  
Mueller Thomas                  55                48              17   120  
Robben Arjen                    20                54              17    91